In [1]:
import os
import sys
import pandas as pd
import numpy as np
import PIL

seed = 16
np.random.seed(seed)

from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator


c:\users\garrick\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Purpose of this notebook is to build/test working model using a smaller subset of classes and data to minimize iteration time and to test CovNets of varying size before running on broader dataset.

In [2]:
#check using system GPU for processing

from tensorflow.python.client import device_lib
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]="0" #for training on gpu
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10183346064960334723
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1493781708
locality {
  bus_id: 1
}
incarnation: 45552984422354421
physical_device_desc: "device: 0, name: GeForce GTX 860M, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [3]:
# copied over the train, validate and test sets for 5 randomly selected breeds

Given the subset, I copied over their respesctive train/validate/test image folders from the broader image data set.  I maintained the full size of each train, val and test set.  

In [4]:
os.chdir('C:\\Users\\Garrick\Documents\\Springboard\\Capstone Project 2\\datasets_subset1')

In [5]:
train_datagen = ImageDataGenerator(rotation_range=15, shear_range=0.1, channel_shift_range=20,
                                    width_shift_range=0.1,  height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True,
                                    fill_mode='nearest', rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 25

train_generator = train_datagen.flow_from_directory('subset_train', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=batch_size)

validation_generator = validation_datagen.flow_from_directory('subset_val', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=batch_size)


test_generator = test_datagen.flow_from_directory('subset_test', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=batch_size)

# reminder to self... flow_from_directory infers the class labels

Found 400 images belonging to 5 classes.
Found 100 images belonging to 5 classes.
Found 389 images belonging to 5 classes.


In [6]:
# importing keras modules and setting up a few parameters, instantiating early stopping

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
import keras.utils
from keras import backend as K
K.set_image_dim_ordering('tf')

from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=2)


In [7]:
import tensorflow as tf
tf_config = tf.ConfigProto()
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.99
# tf_config.gpu_options.allow_growth = True **this causes python to crash, error: las.cc:444] failed to create cublas handle: CUBLAS_STATUS_ALLOC_FAILED
sess = tf.Session(config=tf_config)

In [8]:
input_shape = (224,224, 3)
num_classes = 5

# will create a few different models.... initial base model 

base_model = Sequential()
base_model.add(Conv2D(64, (11, 11), strides=4, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
base_model.add(MaxPooling2D(pool_size=(2, 2)))

base_model.add(Conv2D(64, (4, 4), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
base_model.add(MaxPooling2D(pool_size=(2, 2)))

base_model.add(Conv2D(64, (4, 4), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
base_model.add(Flatten())

base_model.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
base_model.add(Dropout(0.2))
base_model.add(Dense(num_classes, activation='softmax'))
    
# Compile model
epochs = 10
lrate = 0.003
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
base_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(base_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 54, 54, 64)        23296     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        65600     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 2, 64)          65600     
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
__________

In [17]:
# train base_model

base_model.fit_generator(train_generator, validation_data=validation_generator,
                    steps_per_epoch=100, epochs=epochs, callbacks=[early_stopping])

Epoch 1/10
100/100 [==============================] - 39s 387ms/step - loss: 1.6262 - acc: 0.1252 - val_loss: 1.6064 - val_acc: 0.2000
Epoch 2/10
100/100 [==============================] - 47s 475ms/step - loss: 1.6141 - acc: 0.1580 - val_loss: 1.5952 - val_acc: 0.2600
Epoch 3/10
100/100 [==============================] - 51s 512ms/step - loss: 1.6025 - acc: 0.2452 - val_loss: 1.5641 - val_acc: 0.2800
Epoch 4/10
100/100 [==============================] - 49s 486ms/step - loss: 1.5770 - acc: 0.2528 - val_loss: 1.5359 - val_acc: 0.2600
Epoch 5/10
100/100 [==============================] - 49s 491ms/step - loss: 1.5524 - acc: 0.2804 - val_loss: 1.5027 - val_acc: 0.2900
Epoch 6/10
100/100 [==============================] - 49s 489ms/step - loss: 1.5197 - acc: 0.3080 - val_loss: 1.5119 - val_acc: 0.3200
Epoch 7/10
100/100 [==============================] - 49s 489ms/step - loss: 1.5007 - acc: 0.3052 - val_loss: 1.4867 - val_acc: 0.3000
Epoch 8/10
100/100 [==============================] - 4

In [8]:
# same model, more epochs (10 -> 50) and fewere steps per epoch, prior model params saw train and validate accuracies double, expecting the model to be within 80% acc

base_model.fit_generator(train_generator, validation_data=validation_generator,
                    steps_per_epoch=50, epochs=25, callbacks=[early_stopping])

NameError: name 'base_model' is not defined

Looks like we are increasing in accuracy with each successive epoch, perhaps need to train for more epochs. Let's test a deeper network with the same amount of epochs and see if we can begin with a better first epoch accuracy of 15%.

In [10]:
# taking the base model and adding more hidden layers

deep_model = Sequential()

deep_model = Sequential()
deep_model.add(Conv2D(64, (11, 11), strides=4, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
deep_model.add(MaxPooling2D(pool_size=(2, 2)))

deep_model.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model.add(MaxPooling2D(pool_size=(2, 2)))

deep_model.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model.add(MaxPooling2D(pool_size=(2, 2)))

deep_model.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model.add(Flatten())

deep_model.add(Dense(288, activation='relu', kernel_constraint=maxnorm(3)))
deep_model.add(Dropout(0.2))
deep_model.add(Dense(num_classes, activation='softmax'))
    
# Compile model
epochs = 10
lrate = 0.003
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
deep_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(deep_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 54, 54, 64)        23296     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 27, 27, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 25, 25, 32)        18464     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 10, 10, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 3, 3, 32)          9248      
__________

In [11]:
# train deeper model

deep_model.fit_generator(train_generator, validation_data=validation_generator,
                    steps_per_epoch=100, epochs=epochs, callbacks=[early_stopping])

Epoch 1/10
100/100 [==============================] - 50s 496ms/step - loss: 1.6209 - acc: 0.1364 - val_loss: 1.6092 - val_acc: 0.1800
Epoch 2/10
100/100 [==============================] - 49s 486ms/step - loss: 1.6205 - acc: 0.1112 - val_loss: 1.6094 - val_acc: 0.2000
Epoch 3/10
100/100 [==============================] - 50s 497ms/step - loss: 1.6122 - acc: 0.1644 - val_loss: 1.6038 - val_acc: 0.2700
Epoch 4/10
100/100 [==============================] - 50s 497ms/step - loss: 1.6074 - acc: 0.2400 - val_loss: 1.5855 - val_acc: 0.3000
Epoch 5/10
100/100 [==============================] - 50s 495ms/step - loss: 1.5968 - acc: 0.2500 - val_loss: 1.5488 - val_acc: 0.3400
Epoch 6/10
100/100 [==============================] - 51s 507ms/step - loss: 1.5761 - acc: 0.2812 - val_loss: 1.5453 - val_acc: 0.3100
Epoch 7/10
100/100 [==============================] - 51s 507ms/step - loss: 1.5551 - acc: 0.3064 - val_loss: 1.4878 - val_acc: 0.3700
Epoch 8/10
100/100 [==============================] - 5

In [12]:
# looks like the deeper model overfits the training data, but performs better on the validation data... let's train for more epochs

deep_model.fit_generator(train_generator, validation_data=validation_generator,
                    steps_per_epoch=100, epochs=50, callbacks=[early_stopping])

Epoch 1/50
100/100 [==============================] - 48s 481ms/step - loss: 1.4756 - acc: 0.3436 - val_loss: 1.3821 - val_acc: 0.4700
Epoch 2/50
100/100 [==============================] - 49s 487ms/step - loss: 1.4540 - acc: 0.3536 - val_loss: 1.3937 - val_acc: 0.4300
Epoch 3/50
100/100 [==============================] - 49s 488ms/step - loss: 1.4160 - acc: 0.3924 - val_loss: 1.3869 - val_acc: 0.4100


In [15]:
# deeper model ran into early stopping on the validation set

# lets try base model with more epochs

base_model.fit_generator(train_generator, validation_data=validation_generator,
                    steps_per_epoch=100, epochs=25, callbacks=[early_stopping])

Epoch 1/25
100/100 [==============================] - 43s 431ms/step - loss: 1.1137 - acc: 0.5544 - val_loss: 1.4292 - val_acc: 0.4400
Epoch 2/25
100/100 [==============================] - 48s 480ms/step - loss: 1.0815 - acc: 0.5672 - val_loss: 1.4291 - val_acc: 0.4500
Epoch 3/25
100/100 [==============================] - 49s 488ms/step - loss: 1.0484 - acc: 0.5800 - val_loss: 1.3839 - val_acc: 0.4900
Epoch 4/25
100/100 [==============================] - 49s 486ms/step - loss: 1.0520 - acc: 0.5760 - val_loss: 1.3593 - val_acc: 0.4600
Epoch 5/25
100/100 [==============================] - 48s 483ms/step - loss: 1.0108 - acc: 0.5976 - val_loss: 1.5123 - val_acc: 0.4800
Epoch 6/25
100/100 [==============================] - 51s 511ms/step - loss: 0.9848 - acc: 0.6028 - val_loss: 1.4050 - val_acc: 0.5000


In [38]:
# deep model with Adam optimizer

# taking the base model and adding more hidden layers



deep_model_Adam = Sequential()

deep_model_Adam = Sequential()
deep_model_Adam.add(Conv2D(64, (11, 11), strides=4, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
deep_model_Adam.add(MaxPooling2D(pool_size=(2, 2)))

deep_model_Adam.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model_Adam.add(MaxPooling2D(pool_size=(2, 2)))

deep_model_Adam.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model_Adam.add(MaxPooling2D(pool_size=(2, 2)))

deep_model_Adam.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model_Adam.add(Flatten())

deep_model_Adam.add(Dense(288, activation='relu', kernel_constraint=maxnorm(3)))
deep_model_Adam.add(Dropout(0.2))
deep_model_Adam.add(Dense(num_classes, activation='softmax'))
    
# Compile model
adam_op = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
deep_model_Adam.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(deep_model_Adam.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 54, 54, 64)        23296     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 27, 27, 64)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 25, 25, 32)        18464     
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 10, 10, 32)        9248      
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 3, 3, 32)          9248      
__________

In [20]:
deep_model_Adam.fit_generator(train_generator, validation_data=validation_generator,
                    steps_per_epoch=100, epochs=25, callbacks=[early_stopping])

Epoch 1/25
100/100 [==============================] - 52s 519ms/step - loss: 1.6144 - acc: 0.1948 - val_loss: 1.6012 - val_acc: 0.2600
Epoch 2/25
100/100 [==============================] - 53s 533ms/step - loss: 1.5928 - acc: 0.2372 - val_loss: 1.5298 - val_acc: 0.3700
Epoch 3/25
100/100 [==============================] - 52s 524ms/step - loss: 1.5182 - acc: 0.3160 - val_loss: 1.4673 - val_acc: 0.3400
Epoch 4/25
100/100 [==============================] - 53s 534ms/step - loss: 1.4677 - acc: 0.3624 - val_loss: 1.3619 - val_acc: 0.4200
Epoch 5/25
100/100 [==============================] - 53s 529ms/step - loss: 1.4201 - acc: 0.3836 - val_loss: 1.3594 - val_acc: 0.3600
Epoch 6/25
100/100 [==============================] - 52s 519ms/step - loss: 1.3792 - acc: 0.3968 - val_loss: 1.3576 - val_acc: 0.4000
Epoch 7/25
100/100 [==============================] - 54s 538ms/step - loss: 1.3895 - acc: 0.3828 - val_loss: 1.3431 - val_acc: 0.4600
Epoch 8/25
100/100 [==============================] - 5

In [22]:
# tweaked deep model w/ Adam optimizer.  Deeper network topology near the input (less convolution than prior models), more FC nodes

deep_model_Adam_2 = Sequential()

deep_model_Adam_2 = Sequential()
deep_model_Adam_2.add(Conv2D(64, (8, 8), strides=2, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
deep_model_Adam_2.add(MaxPooling2D(pool_size=(2, 2)))

deep_model_Adam_2.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model_Adam_2.add(MaxPooling2D(pool_size=(2, 2)))

deep_model_Adam_2.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model_Adam_2.add(MaxPooling2D(pool_size=(2, 2)))

deep_model_Adam_2.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model_Adam_2.add(Flatten())

deep_model_Adam_2.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
deep_model_Adam_2.add(Dropout(0.2))
deep_model_Adam_2.add(Dense(num_classes, activation='softmax'))
    
# Compile model
adam_op = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
deep_model_Adam_2.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(deep_model_Adam_2.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 109, 109, 64)      12352     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 52, 52, 32)        18464     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 10, 10, 32)        9248      
__________

In [18]:
deep_model_Adam_2.fit_generator(train_generator, validation_data=validation_generator,
                    steps_per_epoch=50, epochs=50, callbacks=[early_stopping])

Epoch 1/50
50/50 [==============================] - 25s 494ms/step - loss: 1.6373 - acc: 0.1040 - val_loss: 1.6094 - val_acc: 0.2100
Epoch 2/50
50/50 [==============================] - 27s 543ms/step - loss: 1.6099 - acc: 0.1312 - val_loss: 1.6093 - val_acc: 0.2700
Epoch 3/50
50/50 [==============================] - 26s 520ms/step - loss: 1.6101 - acc: 0.1248 - val_loss: 1.6093 - val_acc: 0.1800
Epoch 4/50
50/50 [==============================] - 25s 496ms/step - loss: 1.6100 - acc: 0.1440 - val_loss: 1.6087 - val_acc: 0.2100
Epoch 5/50
50/50 [==============================] - 25s 504ms/step - loss: 1.6100 - acc: 0.1712 - val_loss: 1.6093 - val_acc: 0.2100
Epoch 6/50
50/50 [==============================] - 29s 579ms/step - loss: 1.6124 - acc: 0.1560 - val_loss: 1.6094 - val_acc: 0.2000


Deeper topology not necessarily better and is over-fitting.  

In [15]:
# tweaked deep model w/ RMSProp optimizer again with Deeper network topology near the input (less convolution than prior models), more FC nodes

deep_model_RMS = Sequential()

deep_model_RMS = Sequential()
deep_model_RMS.add(Conv2D(64, (8, 8), strides=2, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
deep_model_RMS.add(MaxPooling2D(pool_size=(2, 2)))

deep_model_RMS.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model_RMS.add(MaxPooling2D(pool_size=(2, 2)))

deep_model_RMS.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model_RMS.add(MaxPooling2D(pool_size=(2, 2)))

deep_model_RMS.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model_RMS.add(Flatten())

deep_model_RMS.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
deep_model_RMS.add(Dropout(0.2))
deep_model_RMS.add(Dense(num_classes, activation='softmax'))
    
# Compile model
deep_model_RMS.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(deep_model_RMS.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 109, 109, 64)      12352     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 52, 52, 32)        18464     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 10, 10, 32)        9248      
__________

In [16]:
deep_model_RMS.fit_generator(train_generator, validation_data=validation_generator,
                    steps_per_epoch=100, epochs=50, callbacks=[early_stopping])

Epoch 1/50
100/100 [==============================] - 50s 502ms/step - loss: 1.8105 - acc: 0.0744 - val_loss: 1.6091 - val_acc: 0.2300
Epoch 2/50
100/100 [==============================] - 50s 498ms/step - loss: 1.6849 - acc: 0.1940 - val_loss: 1.5901 - val_acc: 0.2900
Epoch 3/50
100/100 [==============================] - 51s 514ms/step - loss: 1.7109 - acc: 0.2660 - val_loss: 1.5100 - val_acc: 0.3800
Epoch 4/50
100/100 [==============================] - 48s 482ms/step - loss: 1.6232 - acc: 0.3440 - val_loss: 1.5153 - val_acc: 0.3700
Epoch 5/50
100/100 [==============================] - 49s 495ms/step - loss: 1.4819 - acc: 0.3592 - val_loss: 1.4684 - val_acc: 0.3900
Epoch 6/50
100/100 [==============================] - 48s 484ms/step - loss: 1.4970 - acc: 0.3952 - val_loss: 1.4337 - val_acc: 0.4300
Epoch 7/50
100/100 [==============================] - 52s 524ms/step - loss: 1.3743 - acc: 0.4148 - val_loss: 1.4509 - val_acc: 0.3700
Epoch 8/50
100/100 [==============================] - 4

In [19]:
# so more layers doesn't work.... let us keep the standard 3 CONV layers and widen the toplogy

wide_model = Sequential()
wide_model.add(Conv2D(32, (3, 3), strides=1, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
wide_model.add(MaxPooling2D(pool_size=(2, 2)))

wide_model.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model.add(MaxPooling2D(pool_size=(2, 2)))

wide_model.add(Conv2D(32, (3, 3), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model.add(MaxPooling2D(pool_size=(2, 2)))
wide_model.add(Flatten())

wide_model.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
wide_model.add(Dropout(0.2))
wide_model.add(Dense(num_classes, activation='softmax'))
    
# Compile model
epochs = 10
wide_model.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(wide_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 109, 109, 32)      9248      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 26, 26, 32)        9248      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 5408)              0         
__________

In [14]:
wide_model.fit_generator(train_generator, validation_data=validation_generator,
                    steps_per_epoch=50, epochs=50, callbacks=[early_stopping])

Epoch 1/50
50/50 [==============================] - 30s 602ms/step - loss: 1.6786 - acc: 0.1352 - val_loss: 1.5989 - val_acc: 0.2400
Epoch 2/50
50/50 [==============================] - 23s 467ms/step - loss: 1.6084 - acc: 0.2248 - val_loss: 1.5867 - val_acc: 0.3300
Epoch 3/50
50/50 [==============================] - 24s 475ms/step - loss: 1.6061 - acc: 0.2192 - val_loss: 1.5725 - val_acc: 0.2800
Epoch 4/50
50/50 [==============================] - 24s 480ms/step - loss: 1.6008 - acc: 0.2120 - val_loss: 1.5623 - val_acc: 0.2800
Epoch 5/50
50/50 [==============================] - 24s 481ms/step - loss: 1.5650 - acc: 0.3064 - val_loss: 1.5225 - val_acc: 0.3300
Epoch 6/50
50/50 [==============================] - 24s 477ms/step - loss: 1.5702 - acc: 0.2808 - val_loss: 1.4849 - val_acc: 0.3800
Epoch 7/50
50/50 [==============================] - 24s 483ms/step - loss: 1.5391 - acc: 0.3504 - val_loss: 1.4880 - val_acc: 0.3700
Epoch 8/50
50/50 [==============================] - 24s 476ms/step - 

In [1]:
# wider doesn't necessarily work... however, slowing the learning rate seems to having a positive impact. same model as above, decrease LR

wide_model_slow_learn = Sequential()
wide_model_slow_learn.add(Conv2D(32, (3, 3), strides=1, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(MaxPooling2D(pool_size=(2, 2)))

wide_model_slow_learn.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(MaxPooling2D(pool_size=(2, 2)))

wide_model_slow_learn.add(Conv2D(32, (3, 3), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(MaxPooling2D(pool_size=(2, 2)))
wide_model_slow_learn.add(Flatten())

wide_model_slow_learn.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(Dropout(0.2))
wide_model_slow_learn.add(Dense(num_classes, activation='softmax'))
    
# Compile model

adam_op = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
wide_model_slow_learn.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(wide_model_slow_learn.summary())


NameError: name 'Sequential' is not defined

In [28]:
wide_model_slow_learn.fit_generator(train_generator, validation_data=validation_generator,
                    steps_per_epoch=50, epochs=50, callbacks=[early_stopping])

Epoch 1/50
50/50 [==============================] - 25s 507ms/step - loss: 1.6161 - acc: 0.1824 - val_loss: 1.6017 - val_acc: 0.2500
Epoch 2/50
50/50 [==============================] - 25s 498ms/step - loss: 1.6093 - acc: 0.2168 - val_loss: 1.5971 - val_acc: 0.2300
Epoch 3/50
50/50 [==============================] - 25s 495ms/step - loss: 1.6005 - acc: 0.2424 - val_loss: 1.5925 - val_acc: 0.2400
Epoch 4/50
50/50 [==============================] - 24s 488ms/step - loss: 1.5992 - acc: 0.2416 - val_loss: 1.5873 - val_acc: 0.2500
Epoch 5/50
50/50 [==============================] - 25s 492ms/step - loss: 1.5852 - acc: 0.2912 - val_loss: 1.5849 - val_acc: 0.2500
Epoch 6/50
50/50 [==============================] - 25s 503ms/step - loss: 1.5902 - acc: 0.2728 - val_loss: 1.5799 - val_acc: 0.2800
Epoch 7/50
50/50 [==============================] - 25s 507ms/step - loss: 1.5895 - acc: 0.2640 - val_loss: 1.5757 - val_acc: 0.2800
Epoch 8/50
50/50 [==============================] - 25s 493ms/step - 

In [29]:
# it appears a slower learning rate might be key in allowing prior models to train for more epochs... 
# let's try a few earlier models with a decreased learning rate

In [39]:
adam_op = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
deep_model_Adam.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(deep_model_Adam.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 54, 54, 64)        23296     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 27, 27, 64)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 25, 25, 32)        18464     
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 10, 10, 32)        9248      
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 3, 3, 32)          9248      
__________

In [40]:
deep_model_Adam.fit_generator(train_generator, validation_data=validation_generator,
                    steps_per_epoch=50, epochs=50, callbacks=[early_stopping])

Epoch 1/50
50/50 [==============================] - 25s 499ms/step - loss: 1.6217 - acc: 0.1968 - val_loss: 1.6168 - val_acc: 0.1800
Epoch 2/50
50/50 [==============================] - 26s 514ms/step - loss: 1.6266 - acc: 0.1760 - val_loss: 1.6126 - val_acc: 0.1900
Epoch 3/50
50/50 [==============================] - 27s 534ms/step - loss: 1.6167 - acc: 0.1840 - val_loss: 1.6093 - val_acc: 0.2100
Epoch 4/50
50/50 [==============================] - 26s 515ms/step - loss: 1.6085 - acc: 0.2272 - val_loss: 1.6074 - val_acc: 0.2300
Epoch 5/50
50/50 [==============================] - 26s 529ms/step - loss: 1.6126 - acc: 0.2000 - val_loss: 1.6057 - val_acc: 0.2300
Epoch 6/50
50/50 [==============================] - 25s 506ms/step - loss: 1.6127 - acc: 0.2008 - val_loss: 1.6047 - val_acc: 0.2400
Epoch 7/50
50/50 [==============================] - 25s 490ms/step - loss: 1.5997 - acc: 0.2576 - val_loss: 1.6034 - val_acc: 0.2300
Epoch 8/50
50/50 [==============================] - 24s 487ms/step - 

In [35]:
# let's try tye base model w/ decreased learning rate and Adam optimizer (vs. SGD)

base_model.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(base_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 54, 54, 64)        23296     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        65600     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 2, 64)          65600     
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
__________

In [36]:
base_model.fit_generator(train_generator, validation_data=validation_generator,
                    steps_per_epoch=50, epochs=50, callbacks=[early_stopping])

Epoch 1/50
50/50 [==============================] - 20s 394ms/step - loss: 1.6114 - acc: 0.1944 - val_loss: 1.6070 - val_acc: 0.2000
Epoch 2/50
50/50 [==============================] - 25s 503ms/step - loss: 1.6071 - acc: 0.2216 - val_loss: 1.6047 - val_acc: 0.2300
Epoch 3/50
50/50 [==============================] - 25s 499ms/step - loss: 1.6029 - acc: 0.2496 - val_loss: 1.6041 - val_acc: 0.2500
Epoch 4/50
50/50 [==============================] - 25s 503ms/step - loss: 1.6034 - acc: 0.2416 - val_loss: 1.6025 - val_acc: 0.2700
Epoch 5/50
50/50 [==============================] - 25s 507ms/step - loss: 1.6035 - acc: 0.2352 - val_loss: 1.6012 - val_acc: 0.2400
Epoch 6/50
50/50 [==============================] - 28s 554ms/step - loss: 1.6022 - acc: 0.2616 - val_loss: 1.6001 - val_acc: 0.3000
Epoch 7/50
50/50 [==============================] - 28s 566ms/step - loss: 1.5990 - acc: 0.2720 - val_loss: 1.5991 - val_acc: 0.2800
Epoch 8/50
50/50 [==============================] - 27s 547ms/step - 

So far the wider model (with less stride, smaller Convolution filter) w/ more trainable parameters and the simple base model appear to perform the best on validation data.  

In [41]:
# let's test on these iterations of the base, deep and wide models

base_scores = base_model.evaluate_generator(test_generator, steps=25)
print("Accuracy: %.2f%%" % (base_scores[1]*100))

Accuracy: 43.16%


In [42]:
deep_model_Adam_scores = deep_model_Adam.evaluate_generator(test_generator, steps=25)
print("Accuracy: %.2f%%" % (deep_model_Adam_scores[1]*100))

Accuracy: 36.64%


In [43]:
wide_model_slow_learn_scores = wide_model_slow_learn.evaluate_generator(test_generator, steps=25)
print("Accuracy: %.2f%%" % (wide_model_slow_learn_scores[1]*100))

Accuracy: 39.58%


Next steps... pick top 2 or 3 models and test and note which performs best.  use top 2-3 on broader image data set (simple_CNN notebook)

In [44]:
# saving models and weights just in case... will need to retrain on broader image sets anyways
base_model.save('subset_base_model.h5')
deep_model_Adam.save('subset_deep_model_Adam.h5')
wide_model_slow_learn.save('subset_wide_model_slow_learn.h5')